In [2]:
# https://raw.githubusercontent.com/theJollySin/mazelib/master/mazelib/mazelib.py

In [3]:
from random import randrange


class Maze(object):
    """ This is a master object meant to hold a rectangular, 2D maze.
    This object includes the methods used to generate and solve the maze,
    as well as the start and end points.
    """

    def __init__(self):
        self.generator = None
        self.grid = None
        self.start = None
        self.end = None
        self.transmuters = []
        self.solver = None
        self.solutions = None
        self.prune = True

    def generate(self):
        """ public method to generate a new maze, and handle some clean-up

        Returns: None
        """
        assert not (self.generator is None), 'No maze-generation algorithm has been set.'

        self.grid = self.generator.generate()
        self.start = None
        self.end = None
        self.solutions = None

    def generate_entrances(self, start_outer=True, end_outer=True):
        """ Generate maze entrances. Entrances can be on the walls, or inside the maze.

        Args:
            start_outer (bool): Do you want the start of the maze to be on an outer wall?
            end_outer (bool): Do you want the end of the maze to be on an outer wall?
        Returns: None
        """
        if start_outer and end_outer:
            self._generate_outer_entrances()
        elif not start_outer and not end_outer:
            self._generate_inner_entrances()
        elif start_outer:
            self.start, self.end = self._generate_opposite_entrances()
        else:
            self.end, self.start = self._generate_opposite_entrances()

        # the start and end shouldn't be right next to each other
        if abs(self.start[0] - self.end[0]) + abs(self.start[1] - self.end[1]) < 2:
            self.generate_entrances(start_outer, end_outer)

    def _generate_outer_entrances(self):
        """ Generate maze entrances, along the outer walls.

        Returns: None
        """
        H = self.grid.shape[0]
        W = self.grid.shape[1]

        start_side = randrange(4)

        # maze entrances will be on opposite sides of the maze.
        if start_side == 0:
            self.start = (0, randrange(1, W, 2))  # North
            self.end = (H - 1, randrange(1, W, 2))
        elif start_side == 1:
            self.start = (H - 1, randrange(1, W, 2))  # South
            self.end = (0, randrange(1, W, 2))
        elif start_side == 2:
            self.start = (randrange(1, H, 2), 0)  # West
            self.end = (randrange(1, H, 2), W - 1)
        else:
            self.start = (randrange(1, H, 2), W - 1)  # East
            self.end = (randrange(1, H, 2), 0)

    def _generate_inner_entrances(self):
        """ Generate maze entrances, randomly within the maze.

        Returns: None
        """
        H, W = self.grid.shape

        self.start = (randrange(1, H, 2), randrange(1, W, 2))
        end = (randrange(1, H, 2), randrange(1, W, 2))

        # make certain the start and end points aren't the same
        while end == self.start:
            end = (randrange(1, H, 2), randrange(1, W, 2))

        self.end = end

    def _generate_opposite_entrances(self):
        """ Generate one inner and one outer entrance.

        Returns:
            tuple: start cell, end cell
        """
        H, W = self.grid.shape

        start_side = randrange(4)

        # pick a side for the outer maze entrance
        if start_side == 0:
            first = (0, randrange(1, W, 2))  # North
        elif start_side == 1:
            first = (H - 1, randrange(1, W, 2))  # South
        elif start_side == 2:
            first = (randrange(1, H, 2), 0)  # West
        else:
            first = (randrange(1, H, 2), W - 1)  # East

        # create an inner maze entrance
        second = (randrange(1, H, 2), randrange(1, W, 2))

        return (first, second)

    def generate_monte_carlo(self, repeat, entrances=3, difficulty=1.0, reducer=len):
        """ Use the Monte Carlo method to generate a maze of defined difficulty.

        This method assumes the generator and solver algorithms are already set.

        1. Generate a maze.
        2. For each maze, generate a series of entrances.
        3. To eliminate boring entrance choices, select only the entrances
            that yield the longest solution to a given maze.
        4. Repeat steps 1 through 3 for several mazes.
        5. Order the mazes based on a reduction function applied to their maximal
            solutions. By default, this reducer will return the solution length.
        6. Based on the 'difficulty' parameter, select one of the mazes.

        Args:
            repeat (int): How many mazes do you want to generate?
            entrances (int): How many different entrance combinations do you want to try?
            difficulty (float): How difficult do you want the final maze to be (zero to one).
            reducer (function): How do you want to determine solution difficulty (default is length).
        Returns: None
        """
        assert (difficulty >= 0.0 and difficulty <= 1.0), 'Maze difficulty must be between 0 to 1.'

        # generate different mazes
        mazes = []
        for _ in range(repeat):
            self.generate()
            this_maze = []

            # for each maze, generate different entrances, and solve
            for _ in range(entrances):
                self.generate_entrances()
                self.solve()
                this_maze.append({'grid': self.grid,
                                  'start': self.start,
                                  'end': self.end,
                                  'solutions': self.solutions})

            # for each maze, find the longest solution
            mazes.append(max(this_maze, key=lambda k: len(k['solutions'])))

        # sort the mazes by the length of their solution
        mazes = sorted(mazes, key=lambda k: reducer(k['solutions'][0]))

        # based on optional parameter, choose the maze of the correct difficulty
        posi = int((len(mazes) - 1) * difficulty)

        # save final results of Monte Carlo Simulations to this object
        self.grid = mazes[posi]['grid']
        self.start = mazes[posi]['start']
        self.end = mazes[posi]['end']
        self.solutions = mazes[posi]['solutions']

    def transmute(self):
        """ transmute an existing maze grid

        Returns: None
        """
        assert not (self.grid is None), 'No maze grid yet exists to transmute.'

        for transmuter in self.transmuters:
            transmuter.transmute(self.grid, self.start, self.end)

    def solve(self):
        """ public method to solve a new maze, if possible

        Returns: None
        """
        assert not (self.solver is None), 'No maze-solving algorithm has been set.'
        assert not (self.start is None) and not (self.end is None), \
            'Start and end times must be set first.'

        self.solutions = self.solver.solve(self.grid, self.start, self.end)
        if self.prune:
            self.solutions = self.solver.prune_solutions(self.solutions)

    def tostring(self, entrances=False, solutions=False):
        """ Return a string representation of the maze.
        This can also display the maze entrances/solutions IF they already exist.

        Args:
            entrances (bool): Do you want to show the entrances of the maze?
            solutions (bool): Do you want to show the solution to the maze?
        Returns:
            str: string representation of the maze
        """
        if self.grid is None:
            return ''

        # build the walls of the grid
        txt = []
        for row in self.grid:
            txt.append(''.join(['#' if cell else ' ' for cell in row]))

        # insert the start and end points
        if entrances and self.start and self.end:
            r, c = self.start
            txt[r] = txt[r][:c] + 'S' + txt[r][c+1:]
            r, c = self.end
            txt[r] = txt[r][:c] + 'E' + txt[r][c+1:]

        # if extant, insert the solution path
        if solutions and self.solutions:
            for r, c in self.solutions[0]:
                txt[r] = txt[r][:c] + '+' + txt[r][c+1:]

        return '\n'.join(txt)

    def __str__(self):
        """ display maze walls, entrances, and solutions, if available

        Returns:
            str: string representation of the maze
        """
        return self.tostring(True, True)

    def __repr__(self):
        """ display maze walls, entrances, and solutions, if available

        Returns:
            str: string representation of the maze
        """
        return self.__str__()


In [4]:
# https://raw.githubusercontent.com/theJollySin/mazelib/master/mazelib/generate/MazeGenAlgo.py

In [5]:

import abc
import numpy as np
from numpy.random import shuffle


class MazeGenAlgo:
    __metaclass__ = abc.ABCMeta

    def __init__(self, h, w):
        assert (w >= 3 and h >= 3), 'Mazes cannot be smaller than 3x3.'
        self.h = h
        self.w = w
        self.H = (2 * self.h) + 1
        self.W = (2 * self.w) + 1

    @abc.abstractmethod
    def generate(self):
        return None

    """ All of the methods below this are helper methods,
    common to many maze-generating algorithms.
    """

    def _find_neighbors(self, r, c, grid, is_wall=False):
        """ Find all the grid neighbors of the current position; visited, or not.

        Args:
            r (int): row of cell of interest
            c (int): column of cell of interest
            grid (np.array): 2D maze grid
            is_wall (bool): Are we looking for neighbors that are walls, or open cells?
        Returns:
            list: all neighboring cells that match our request
        """
        ns = []

        if r > 1 and grid[r - 2][c] == is_wall:
            ns.append((r - 2, c))
        if r < self.H - 2 and grid[r + 2][c] == is_wall:
            ns.append((r + 2, c))
        if c > 1 and grid[r][c - 2] == is_wall:
            ns.append((r, c - 2))
        if c < self.W - 2 and grid[r][c + 2] == is_wall:
            ns.append((r, c + 2))

        shuffle(ns)
        return ns

In [6]:
# https://raw.githubusercontent.com/theJollySin/mazelib/master/mazelib/generate/Prims.py

In [7]:


class Prims(MazeGenAlgo):
    """
    The Algorithm

    1. Choose an arbitrary cell from the grid, and add it to some
        (initially empty) set visited nodes (V).
    2. Randomly select a wall from the grid that connects a cell in
        V with another cell not in V.
    3. Add that wall to the Minimal Spanning Tree (MST), and the edge's other cell to V.
    4. Repeat steps 2 and 3 until V includes every cell in G.
    """

    def __init__(self, h, w):
        super(Prims, self).__init__(h, w)

    def generate(self):
        # create empty grid
        grid = np.empty((self.H, self.W), dtype=np.int8)
        grid.fill(1)

        # choose a random starting position
        current_row = randrange(1, self.H, 2)
        current_col = randrange(1, self.W, 2)
        grid[current_row][current_col] = 0

        # created a weighted list of all vertices connected in the graph
        neighbors = self._find_neighbors(current_row, current_col, grid, True)

        # loop over all current neighbors, until empty
        visited = 1

        while visited < self.h * self.w:
            # find neighbor with lowest weight, make it current
            nn = randrange(len(neighbors))
            current_row, current_col = neighbors[nn]
            visited += 1
            grid[current_row][current_col] = 0
            neighbors = neighbors[:nn] + neighbors[nn + 1:]
            # connect that neighbor to a random neighbor with grid[posi] == 0
            nearest_n0, nearest_n1 = self._find_neighbors(current_row, current_col, grid)[0]
            grid[(current_row + nearest_n0) // 2][(current_col + nearest_n1) // 2] = 0

            # find all unvisited neighbors of current, add them to neighbors
            unvisited = self._find_neighbors(current_row, current_col, grid, True)
            neighbors = list(set(neighbors + unvisited))

        return grid

In [8]:
from random import choice, randrange, shuffle
import numpy as np

RANDOM = 1
SERPENTINE = 2


class DungeonRooms(MazeGenAlgo):
    """
    This is a variation on Hunt-and-Kill where the initial maze has rooms carved out of
    it, instead of being completely flat.

    Optional Parameters

    rooms: List(List(tuple, tuple))
        A list of lists, containing the top-left and bottom-right grid coords of where
        you want rooms to be created. For best results, the corners of each room should
        have odd-numbered coordinates.
    grid: i8[H,W]
        A pre-built maze array filled with one, or many, rooms.
    hunt_order: String ['random', 'serpentine']
        Determines how the next cell to hunt from will be chosen. (default 'random')
    """

    def __init__(self, h0, w0, rooms=None, grid=None, hunt_order='random'):
        # if the user provides a grid, that overrides h & w
        if grid is not None:
            h = (grid.shape[0] - 1) // 2
            w = (grid.shape[1] - 1) // 2
            self.backup_grid = grid.copy()
        else:
            h = h0
            w = w0
            self.backup_grid = np.empty((2 * h + 1, 2 * w + 1), dtype=np.int8)
            self.backup_grid.fill(1)
        self.grid = grid
        self.rooms = rooms
        super(DungeonRooms, self).__init__(h, w)

        # the user can define what order to hunt for the next cell in
        if hunt_order.lower().strip() == 'serpentine':
            self._hunt_order = SERPENTINE
        else:
            self._hunt_order = RANDOM

    def generate(self):
        # define grid and rooms
        self.grid = self.backup_grid.copy()
        self._carve_rooms(self.rooms)

        # select start position for algorithm
        current = self._choose_start()
        self.grid[current[0]][current[1]] = 0

        # perform many random walks, to fill the maze
        num_trials = 0
        while current != (-1, -1):
            self._walk(current)
            current = self._hunt(num_trials)
            num_trials += 1

        # fix any unconnected wall sections
        self.reconnect_maze()

        return self.grid

    def _carve_rooms(self, rooms):
        """ Open up user-defined rooms in a maze. """
        if rooms is None:
            return

        for room in rooms:
            try:
                top_left, bottom_right = room
                self._carve_room(top_left, bottom_right)
                self._carve_door(top_left, bottom_right)
            except Exception:
                # If the user tries to create an invalid room, it is simply ignored.
                pass

    def _carve_room(self, top_left, bottom_right):
        """ Open up a single user-defined room in a maze. """
        for row in range(top_left[0], bottom_right[0] + 1):
            for col in range(top_left[1], bottom_right[1] + 1):
                self.grid[row, col] = 0

    def _carve_door(self, top_left, bottom_right):
        """ Open up a single door in a user-defined room,
        IF that room does not already have a whole wall of doors.
        """
        even_squares = [i for i in list(top_left) + list(bottom_right) if i % 2 == 0]
        if len(even_squares) > 0:
            return

        # find possible doors on all sides of room
        possible_doors = []
        odd_rows = [i for i in range(top_left[0] - 1, bottom_right[0] + 2) if i % 2 == 1]
        odd_cols = [i for i in range(top_left[1] - 1, bottom_right[1] + 2) if i % 2 == 1]

        if top_left[0] > 2:
            possible_doors += zip([top_left[0] - 1] * len(odd_rows), odd_rows)
        if top_left[1] > 2:
            possible_doors += zip(odd_cols, [top_left[1] - 1] * len(odd_cols))
        if bottom_right[0] < self.grid.shape[0] - 2:
            possible_doors += zip([bottom_right[0] + 1] * len(odd_rows), odd_rows)
        if bottom_right[1] < self.grid.shape[1] - 2:
            possible_doors += zip(odd_cols, [bottom_right[1] + 1] * len(odd_cols))

        door = choice(possible_doors)
        self.grid[door[0], door[1]] = 0

    def _walk(self, start):
        """ This is a standard random walk. It must start from a visited cell.
        And it completes when the current cell has no unvisited neighbors.
        """
        if self.grid[start[0], start[1]] == 0:
            current = start
            unvisited_neighbors = self._find_neighbors(current[0], current[1], self.grid, True)

            while len(unvisited_neighbors) > 0:
                neighbor = choice(unvisited_neighbors)
                self.grid[neighbor[0], neighbor[1]] = 0
                self.grid[(neighbor[0] + current[0]) // 2, (neighbor[1] + current[1]) // 2] = 0
                current = neighbor
                unvisited_neighbors = self._find_neighbors(current[0], current[1], self.grid, True)

    def _hunt(self, count):
        """ Based on how this algorithm was configured, choose hunt for the next starting point. """
        if self._hunt_order == SERPENTINE:
            return self._hunt_serpentine(count)
        else:
            return self._hunt_random(count)

    def _hunt_random(self, count):
        """ Select the next cell to walk from, randomly. """
        if count >= (self.H * self.W):
            return (-1, -1)

        return (randrange(1, self.H, 2), randrange(1, self.W, 2))

    def _hunt_serpentine(self, count):
        """ Select the next cell to walk from by cycling through every grid cell in order. """
        cell = (1, 1)
        found = False

        while not found:
            cell = (cell[0], cell[1] + 2)
            if cell[1] > (self.W - 2):
                cell = (cell[0] + 2, 1)
                if cell[0] > (self.H - 2):
                    return (-1, -1)

            if self.grid[cell[0]][cell[1]] == 0 and len(self._find_neighbors(cell[0], cell[1], self.grid, True)) > 0:
                found = True

        return cell

    def _choose_start(self):
        """ Choose a random starting location, that is not already inside a room.
        If no such room exists, the input grid was invalid.
        """
        current = (randrange(1, self.H, 2), randrange(1, self.W, 2))

        LIMIT = self.H * self.W * 2
        num_tries = 1

        # keep looping until you find an unvisited cell
        while num_tries < LIMIT:
            current = (randrange(1, self.H, 2), randrange(1, self.W, 2))
            if self.grid[current[0]][current[1]] == 1:
                return current
            num_tries += 1

        assert num_tries < LIMIT, 'The grid input to DungeonRooms was invalid.'

        return current

    def reconnect_maze(self):
        """ If a maze is not fully connected, open up walls until it is. """
        self._fix_disjoint_passages(self._find_all_passages())

    def _find_all_passages(self):
        """ Place all connected passage cells into a set.
        Disjoint passages will be in different sets.
        """
        passages = []

        # go through all cells in the maze
        for r in range(1, self.grid.shape[0], 2):
            for c in range(1, self.grid.shape[1], 2):
                ns = self._find_unblocked_neighbors((r, c))
                current = set(ns + [(r, c)])

                # determine which passage(s) the current neighbors belong in
                found = False
                for i, passage in enumerate(passages):
                    intersect = current.intersection(passage)
                    if len(intersect) > 0:
                        passages[i] = passages[i].union(current)
                        found = True
                        break

                # the current neighbors might be a disjoint set
                if not found:
                    passages.append(current)

        return self._join_intersecting_sets(passages)

    def _fix_disjoint_passages(self, disjoint_passages):
        """ All passages in a maze should be connected """
        while len(disjoint_passages) > 1:
            found = False
            while not found:
                # randomly select a cell in the first passage
                cell = choice(list(disjoint_passages[0]))
                neighbors = self._find_neighbors(cell[0], cell[1], self.grid)
                # determine if that cell has a neighbor in any other passage
                for passage in disjoint_passages[1:]:
                    intersect = [c for c in neighbors if c in passage]
                    # if so, remove the dividing wall, and combine the two passages
                    if len(intersect) > 0:
                        mid = self._midpoint(intersect[0], cell)
                        self.grid[mid[0], mid[1]] = 0
                        disjoint_passages[0] = disjoint_passages[0].union(passage)
                        disjoint_passages.remove(passage)
                        found = True
                        break

    def _find_unblocked_neighbors(self, posi):
        """ Find all the grid neighbors of the current position; visited, or not.
        """
        r, c = posi
        ns = []

        if r > 1 and self.grid[r-1][c] == False and self.grid[r-2][c] == False:
            ns.append((r-2, c))
        if r < self.grid.shape[0]-2 and self.grid[r+1][c] == False and self.grid[r+2][c] == False:
            ns.append((r+2, c))
        if c > 1 and self.grid[r][c-1] == False and self.grid[r][c-2] == False:
            ns.append((r, c-2))
        if c < self.grid.shape[1]-2 and self.grid[r][c+1] == False and self.grid[r][c+2] == False:
            ns.append((r, c+2))

        shuffle(ns)

        return ns

    def _join_intersecting_sets(self, list_of_sets):
        """ combine sets that have non-zero intersections """
        for i in range(len(list_of_sets) - 1):
            if list_of_sets[i] is None:
                continue

            for j in range(i + 1, len(list_of_sets)):
                if list_of_sets[j] is None:
                    continue
                intersect = list_of_sets[i].intersection(list_of_sets[j])
                if len(intersect) > 0:
                    list_of_sets[i] = list_of_sets[i].union(list_of_sets[j])
                    list_of_sets[j] = None

        return list(filter(lambda l: l is not None, list_of_sets))

    def _midpoint(self, a, b):
        """ Find the wall cell between to passage cells """
        return ((a[0] + b[0]) // 2, (a[1] + b[1]) // 2)

In [9]:


m = Maze()
m.generator = DungeonRooms(5, 15)
m.generate()
output = m.tostring()
print(output)


###############################
#         #   # #         #   #
# # ##### ### # # ####### ### #
# # #   #         #   #     # #
### # ### ########### # # # # #
#   #   # #         #   # #   #
# ##### # # ####### ##### # # #
# #   # # # #       #     # # #
# ### # # # # ### ### ##### # #
#       #   #   #     #     # #
###############################
